In [ ]:
import torch
import evaluate

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
regard_tokenizer = AutoTokenizer.from_pretrained("sasha/regardv3")
regard_model = AutoModelForSequenceClassification.from_pretrained("sasha/regardv3")
regard_classifier = pipeline(
            "text-classification", model=regard_model,batch_size=64, top_k=4, tokenizer=regard_tokenizer, truncation=True, device='cuda'
        )

In [ ]:
from collections import defaultdict
def regard(group):
    group_scores = defaultdict(list)
    group_regard = regard_classifier(group)
    return group_regard

In [ ]:
from tqdm import tqdm
import pickle
with open('./comments.pkl', 'rb') as handle:
    comments = pickle.load(handle)

In [ ]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [ ]:
results = []
with torch.no_grad():
  regard_model.to('cuda')
for chunk in tqdm(chunker(comments,256)):
    results += regard(chunk)

In [ ]:
regard_map = {}
for comment, result in zip(comments, results):
  classification = {l['label']: round(l['score'],2) for l in result}
  regard_map[comment] = classification

In [ ]:
with open('./comment_regard.pkl', 'wb') as handle:
  pickle.dump(regard_map, handle, protocol=pickle.HIGHEST_PROTOCOL)